In [156]:
import numpy as np
import pandas as pd
import tensorflow as tf

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from tensorflow.examples.tutorials.mnist import input_data

In [157]:
LEARNING_RATE = 1e-4

# iterations for training
TRAINING_ITERATIONS = 2500

# avoid overfitting
DROPOUT = 0.5

BATCH_SIZE = 50

VALIDATION_SIZE = 2000

IMAGE_TO_DISPLAY = 10

In [158]:
data = input_data.read_data_sets('../../MNIST_data', one_hot=True)

Extracting ../../MNIST_data/train-images-idx3-ubyte.gz
Extracting ../../MNIST_data/train-labels-idx1-ubyte.gz
Extracting ../../MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ../../MNIST_data/t10k-labels-idx1-ubyte.gz


In [159]:
# # 1. handle data

# images = data.iloc[:, 1:].values
# images = images.astype(np.float)

# images = np.multiply(images, 1.0 / 255.0)

# print (images.shape)

# image_size = images.shape[1]

# image_width = image_height = np.ceil(np.sqrt(image_size)).astype(np.uint8)

# print(image_width, image_height)

# def Display(img):
#     one_image = img.reshape(image_width, image_height)
    
#     plt.axis('off')
#     plt.imshow(one_image, cmap=cm.binary)

# Display(images[IMAGE_TO_DISPLAY + 1])

# # 2. handle label

# label_flat = data[[0]].values.ravel()
# labels_count = np.unique(label_flat).shape[0]

# ## 2.1 one hot encode labels

# # def DenseToOneHot(labels):
# #     num_labels = labels.shape[0]
# #     num_classes = np.unique(label_flat).shape[0]
# #     index_offset = np.arange(num_labels) * num_classes

# #     labels_one_hot = np.zeros((num_labels, num_classes))
# #     labels_one_hot.flat[index_offset + labels.ravel()] = 1
# #     return labels_one_hot

# # labels = DenseToOneHot(label_flat)
# # print labels.shape

# # use sklearn to onhotencoder labels

# from sklearn.preprocessing import OneHotEncoder

# labels_reshape = label_flat.reshape(label_flat.shape[0], 1)

# enc = OneHotEncoder().fit_transform(labels_reshape)

# labels = enc.toarray()

# # 3. split data to train and test data

# from sklearn.cross_validation import train_test_split

# train_images, test_images, train_labels, test_labels = train_test_split(images, labels, random_state=2016, test_size=.4)

In [ ]:
def WeightVariable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def BiasVariabel(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def Conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def MaxPoll2X2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')



# define placeholder for inputs to network
xs = tf.placeholder(tf.float32, [None, 784]) # 28x28
ys = tf.placeholder(tf.float32, [None, 10])
keep_prob = tf.placeholder(tf.float32)
x_image = tf.reshape(xs, [-1, 28, 28, 1])
# print(x_image.shape)  # [n_samples, 28,28,1]

## conv1 layer ##
W_conv1 = WeightVariable([5,5, 1,32]) # patch 5x5, in size 1, out size 32
b_conv1 = BiasVariabel([32])
h_conv1 = tf.nn.relu(Conv2d(x_image, W_conv1) + b_conv1) # output size 28x28x32
h_pool1 = MaxPoll2X2(h_conv1)                                         # output size 14x14x32


W_conv2 = WeightVariable([5, 5, 32, 64])
b_conv2 = BiasVariabel([64])
h_conv2 = tf.nn.relu(Conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = MaxPoll2X2(h_conv2)


W_fc1 = WeightVariable([7*7*64, 1024])
b_fcl = BiasVariabel([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat , W_fc1) + b_fcl)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

W_fc2 = WeightVariable([1024, 10])
b_fc2 = BiasVariabel([10])
y = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

cross_entropy = tf.reduce_mean(-tf.reduce_sum(ys * tf.log(y), reduction_indices=[1]))

train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

sess = tf.Session()
sess.run(tf.initialize_all_variables())

#print batch

for step in range(TRAINING_ITERATIONS):
    batch_xs, batch_ys = data.train.next_batch(100)


    sess.run(train_step, feed_dict={xs: batch_xs, ys: batch_ys, keep_prob: 0.5})

In [ ]:
batch = np.random.randint(images.shape[0], size=BATCH_SIZE)
print labels[batch]